In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.ensemble import RandomForestRegressor

In [2]:
immigration_data_2012_2021 = "Resources/immigration_data_2005_2021.csv"
#state_unemployment = "Resources/emp-unemployment.xls"
#gini = "Resources/gini_index_by_state_2016_2018.xlsx"

In [3]:
inmigration = pd.read_csv(immigration_data_2012_2021)
inmigration.head()

,Year,Region and country of birth,Total Permanent Residents,Percentage,Alabama,Alaska,Arizona,Arkansas,California,Colorado,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,U.S. Armed Services posts,U.S. possessions,U.S. Dependencies,Guam,Puerto Rico
0,2005,Total,1122373,100.00,4200,1525,18988,2698,232023,11977,...,27100,26482,847,7909,321,128,5868,0,0,0
1,2005,"China, People's Republic",69967,6.23,328,92,543,202,17668,765,...,1327,1508,101,593,28,13,184,0,0,0
2,2005,Dominican Republic,27504,2.45,5,42,22,0,82,6,...,90,18,6,39,0,0,2346,0,0,0
3,2005,India,84681,7.54,431,15,739,215,14724,516,...,2776,1747,133,876,0,0,24,0,0,0
4,2005,Iran,13887,1.24,48,4,285,9,7059,131,...,562,318,18,48,0,0,0,0,0,0


## Cleaning country regressor dataset

In [4]:
#Read the csv
countries_raw_metadata = pd.read_csv("Resources/countries_metadata.csv", encoding='latin-1')
display(countries_raw_metadata.head())
print(countries_raw_metadata.shape)

,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,China,CHN,"Population, total",SP.POP.TOTL,667070000,660330000,665770000,682335000,698355000,715185000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,CHN,GDP per capita (current US$),NY.GDP.PCAP.CD,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,CHN,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,CHN,Urban population (% of total population),SP.URB.TOTL.IN.ZS,16.203,16.708,17.226,17.757,18.299,18.086,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,CHN,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,20.86,18.02,37.01,43.37,39.14,37.88,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..


(70, 67)


In [5]:
#Delete the "[YR..]" in the year columns
for column in countries_raw_metadata.columns:
    if "[YR" in column:
        new_column_name = column.split("[")[0].strip() # Extract the part before '[YR' and remove any leading/trailing spaces
        countries_raw_metadata.rename(columns={column: new_column_name}, inplace=True)
countries_raw_metadata.head()

,Country Name,Country Code,Series Name,Series Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,CHN,"Population, total",SP.POP.TOTL,667070000,660330000,665770000,682335000,698355000,715185000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,CHN,GDP per capita (current US$),NY.GDP.PCAP.CD,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,CHN,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,CHN,Urban population (% of total population),SP.URB.TOTL.IN.ZS,16.203,16.708,17.226,17.757,18.299,18.086,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,CHN,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,20.86,18.02,37.01,43.37,39.14,37.88,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..


In [6]:
#Clean the "Country Names" columns 
unique_names = countries_raw_metadata["Country Name"].unique()
unique_names

array(['China', 'United States', 'Iran, Islamic Rep.', 'India',
       'Dominican Republic', 'Mexico', 'United Kingdom', 'Pakistan',
       'Philippines', nan,
       'Data from database: World Development Indicators',
       'Last Updated: 06/29/2023', 'Code', 'SP.POP.TOTL',
       'NY.GDP.PCAP.CD', 'SL.UEM.TOTL.NE.ZS',
       'Unemployment is a key measure to monitor whether a country is on track to achieve the Sustainable Development Goal of promoting sustained',
       'However', 'SP.URB.TOTL.IN.ZS',
       "Percentages urban are the numbers of persons residing in an area defined as ''urban'' per 100 total population. They are calculated by the Statistics Division of the United Nations Department of Economic and Social Affairs. Particular caution should be used in interpreting the figures for percentage urban for different countries.",
       'Countries differ in the way they classify population as "urban" or "rural." The population of a city or metropolitan area depends on the bou

In [7]:
#List the country names we want to keep.
attributes_to_keep = ['China', 'United States', 'Iran, Islamic Rep.', 'India', 'Dominican Republic', 'Mexico', 'United Kingdom', 'Pakistan', 'Philippines']

# Filter the DataFrame to keep only the desired attributes
df_filtered = countries_raw_metadata[countries_raw_metadata['Country Name'].isin(attributes_to_keep)]

# Print the filtered DataFrame
df_filtered.head()

,Country Name,Country Code,Series Name,Series Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,CHN,"Population, total",SP.POP.TOTL,667070000,660330000,665770000,682335000,698355000,715185000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,CHN,GDP per capita (current US$),NY.GDP.PCAP.CD,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,CHN,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,CHN,Urban population (% of total population),SP.URB.TOTL.IN.ZS,16.203,16.708,17.226,17.757,18.299,18.086,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,CHN,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,20.86,18.02,37.01,43.37,39.14,37.88,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..


In [8]:
#Prepare the "Series Name" to move it to the columns, these will be our regressors. 
series_names = df_filtered["Series Name"].unique()
series_names

array(['Population, total', 'GDP per capita (current US$)',
       'Unemployment, total (% of total labor force) (national estimate)',
       'Urban population (% of total population)',
       'Birth rate, crude (per 1,000 people)', 'Gini index'], dtype=object)

In [9]:
df_filtered.loc[df_filtered["Series Name"] == 'Population, total', "Series Name"] = "Population_total"
df_filtered.loc[df_filtered["Series Name"] == 'GDP per capita (current US$)', "Series Name"] = "GDP_per_capita"
df_filtered.loc[df_filtered["Series Name"] == 'Unemployment, total (% of total labor force) (national estimate)', "Series Name"] = "UR"
df_filtered.loc[df_filtered["Series Name"] == 'Urban population (% of total population)', "Series Name"] = "Urban_population"
df_filtered.loc[df_filtered["Series Name"] == 'Birth rate, crude (per 1,000 people)', "Series Name"] = "Birth_rate_crude"
df_filtered.loc[df_filtered["Series Name"] == 'Gini index', "Series Name"] = "Gini_index"
df_filtered.loc[df_filtered["Country Name"] == 'Iran, Islamic Rep.', "Country Name"] = "Iran"
df_filtered

,Country Name,Country Code,Series Name,Series Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,CHN,Population_total,SP.POP.TOTL,667070000,660330000,665770000,682335000,698355000,715185000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,CHN,GDP_per_capita,NY.GDP.PCAP.CD,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,CHN,UR,SL.UEM.TOTL.NE.ZS,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,CHN,Urban_population,SP.URB.TOTL.IN.ZS,16.203,16.708,17.226,17.757,18.299,18.086,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,CHN,Birth_rate_crude,SP.DYN.CBRT.IN,20.86,18.02,37.01,43.37,39.14,37.88,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..
5,China,CHN,Gini_index,SI.POV.GINI,..,..,..,..,..,..,...,39.7,39.2,38.6,38.5,39.1,3.850000e+01,38.2,..,..,..
6,United States,USA,Population_total,SP.POP.TOTL,180671000,183691000,186538000,189242000,191889000,194303000,...,316059947,318386329,320738994,323071755,325122128,3.268382e+08,328329953,331511512,332031554,333287557
7,United States,USA,GDP_per_capita,NY.GDP.PCAP.CD,3007.12344537862,3066.56286916615,3243.84307754988,3374.51517105082,3573.94118474743,3827.52710972039,...,53291.1276891406,55123.8497869046,56762.7294515989,57866.7449341091,59907.754260885,6.282331e+04,65120.3946628653,63528.6343027508,70219.472454115,76398.5917422054
8,United States,USA,UR,SL.UEM.TOTL.NE.ZS,5.5,6.7,5.5,5.7,5.2,4.5,...,7.37,6.17,5.28,4.87,4.36,3.900000e+00,3.67,8.05,5.35,3.65
9,United States,USA,Urban_population,SP.URB.TOTL.IN.ZS,69.996,70.377,70.757,71.134,71.508,71.879,...,81.299,81.483,81.671,81.862,82.058,8.225600e+01,82.459,82.664,82.873,83.084


In [10]:
# Get rid the unnecessary columns
countries_regressors_raw = df_filtered.drop(columns= {"Country Code", "Series Code"} )
countries_regressors_raw.head()

,Country Name,Series Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,Population_total,667070000,660330000,665770000,682335000,698355000,715185000,735400000,754550000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,GDP_per_capita,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,104.324189073855,96.58918279471,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,UR,..,..,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,Urban_population,16.203,16.708,17.226,17.757,18.299,18.086,17.915,17.785,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,Birth_rate_crude,20.86,18.02,37.01,43.37,39.14,37.88,35.05,33.96,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..


In [11]:
#countries_regressors_raw.info()

## Cleaning inmigration csv

In [12]:
migration = inmigration.rename(columns={"Region and country of birth": "Country_origin", "Total Permanent Residents": "Total_inmigrants"})
migration = migration[migration["Country_origin"]!= "Total"]
migration = migration.drop(columns="Percentage")
migration["Country_origin"] = migration["Country_origin"].replace("China, People's Republic", "China")
migration

,Year,Country_origin,Total_inmigrants,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,U.S. Armed Services posts,U.S. possessions,U.S. Dependencies,Guam,Puerto Rico
1,2005,China,69967,328,92,543,202,17668,765,894,...,1327,1508,101,593,28,13,184,0,0,0
2,2005,Dominican Republic,27504,5,42,22,0,82,6,319,...,90,18,6,39,0,0,2346,0,0,0
3,2005,India,84681,431,15,739,215,14724,516,1571,...,2776,1747,133,876,0,0,24,0,0,0
4,2005,Iran,13887,48,4,285,9,7059,131,88,...,562,318,18,48,0,0,0,0,0,0
5,2005,Mexico,161445,569,96,8373,870,63092,2891,316,...,870,2330,30,1054,75,0,129,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,2021,Iran,5734,21,0,86,17,2206,89,39,...,218,187,9,38,0,0,0,0,0,0
149,2021,Mexico,107230,323,33,6859,610,31715,3131,247,...,586,2539,20,949,69,0,0,0,0,39
150,2021,Pakistan,9691,31,0,59,41,1104,52,111,...,784,166,17,57,3,0,0,0,0,0
151,2021,Philippines,27511,152,190,555,129,6478,228,178,...,648,737,69,236,27,0,0,0,300,9


In [13]:
# Getting the DataFrame with the years, total number of inmigrants in the US for a given country and 
# the number of inmigrants of this country in one US state
county_origin_us_df = migration[migration["Country_origin"]=="India"]
country_columns = ["Year","Total_inmigrants","California"]
inmigration_us_df = county_origin_us_df[country_columns]
inmigration_us_df

,Year,Total_inmigrants,California
3,2005,84681,14724
12,2006,61369,14110
21,2007,65353,12820
30,2008,63352,14112
39,2009,57304,12826
48,2010,69162,15099
57,2011,69013,15061
66,2012,66434,13951
75,2013,68458,13735
84,2014,77908,15770


### Prepare the regressor DataFrame per Country

In [14]:
# Filter by country
regressor_by_country = countries_regressors_raw.loc[countries_regressors_raw["Country Name"]== "India"]
regressor_by_country 

,Country Name,Series Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
18,India,Population_total,445954579,456351876,467024193,477933619,489059309,500114346,510992617,521987069,...,1291132063,1307246509,1322866505,1338636340,1354195680,1.369003e+09,1383112050,1396387127,1407563842,1417173173
19,India,GDP_per_capita,83.0351017584916,85.9697041171847,90.2768688575718,101.315164901321,115.487608265464,119.082475942029,89.7575826115815,96.0463297617622,...,1438.05699513067,1559.86377161811,1590.17432166504,1714.2795405846,1957.96981368095,1.974378e+03,2050.16378973235,1913.21972103493,2238.12713873311,2388.62119776762
20,India,UR,..,..,..,..,..,..,..,..,...,..,..,..,..,..,7.650000e+00,6.51,7.86,6.38,4.82
21,India,Urban_population,17.924,18.032,18.218,18.405,18.595,18.785,18.977,19.171,...,32.003,32.384,32.777,33.182,33.6,3.403000e+01,34.472,34.926,35.393,35.872
22,India,Birth_rate_crude,42.506,42.294,42.101,41.907,41.721,41.346,40.918,40.592,...,19.935,19.049,18.765,18.514,17.911,1.765100e+01,17.049,16.572,16.419,..
23,India,Gini_index,..,..,..,..,..,..,..,..,...,..,..,34.7,34.8,35.9,3.460000e+01,35.7,..,..,..


In [15]:
# Drop the Country Name column
drop_countryname = regressor_by_country.drop(columns="Country Name")

# Transpose rows per columns
regressor_by_country_not_cleaned = drop_countryname.transpose()

# Use the first row as the column names
regressor_by_country_not_cleaned.columns = regressor_by_country_not_cleaned.iloc[0]

# Drop the first row (previous integer index) since it's now redundant
regressor_by_country_drop_row = regressor_by_country_not_cleaned.iloc[1:]

# Print the DataFrame after updating the column names and dropping the first row
regressor_by_country_drop_row

Series Name,Population_total,GDP_per_capita,UR,Urban_population,Birth_rate_crude,Gini_index
1960,445954579,83.0351017584916,..,17.924,42.506,..
1961,456351876,85.9697041171847,..,18.032,42.294,..
1962,467024193,90.2768688575718,..,18.218,42.101,..
1963,477933619,101.315164901321,..,18.405,41.907,..
1964,489059309,115.487608265464,..,18.595,41.721,..
...,...,...,...,...,...,...
2018,1369003306.0,1974.37773,7.65,34.03,17.651,34.6
2019,1383112050,2050.16378973235,6.51,34.472,17.049,35.7
2020,1396387127,1913.21972103493,7.86,34.926,16.572,..
2021,1407563842,2238.12713873311,6.38,35.393,16.419,..


In [16]:
columns_regressor_country = regressor_by_country_drop_row.columns
columns_regressor_country

Index(['Population_total', 'GDP_per_capita', 'UR', 'Urban_population',
       'Birth_rate_crude', 'Gini_index'],
      dtype='object', name='Series Name')

In [17]:
regressor_by_country_reset_index = regressor_by_country_drop_row.reset_index().rename(columns={"index":"Year"})
regressor_by_country_reset_index.head()

Series Name,Year,Population_total,GDP_per_capita,UR,Urban_population,Birth_rate_crude,Gini_index
0,1960,445954579,83.0351017584916,..,17.924,42.506,..
1,1961,456351876,85.9697041171847,..,18.032,42.294,..
2,1962,467024193,90.2768688575718,..,18.218,42.101,..
3,1963,477933619,101.315164901321,..,18.405,41.907,..
4,1964,489059309,115.487608265464,..,18.595,41.721,..


In [18]:
regressor_by_country_reset_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Year              63 non-null     object
 1   Population_total  63 non-null     object
 2   GDP_per_capita    63 non-null     object
 3   UR                63 non-null     object
 4   Urban_population  63 non-null     object
 5   Birth_rate_crude  63 non-null     object
 6   Gini_index        63 non-null     object
dtypes: object(7)
memory usage: 3.6+ KB


In [19]:
#Change the data type to integers
regressor_by_country_reset_index["Year"] = regressor_by_country_reset_index["Year"].astype(int)
regressor_by_country_reset_index["Population_total"] = regressor_by_country_reset_index["Population_total"].astype(int)
regressor_by_country_reset_index["GDP_per_capita"] = regressor_by_country_reset_index["GDP_per_capita"].astype(float)
regressor_by_country_reset_index['UR'].replace('..', np.nan, inplace=True)
regressor_by_country_reset_index["UR"] = regressor_by_country_reset_index["UR"].astype(float)
regressor_by_country_reset_index["Urban_population"] = regressor_by_country_reset_index["Urban_population"].astype(float)
regressor_by_country_reset_index['Birth_rate_crude'].replace('..', np.nan, inplace=True)
regressor_by_country_reset_index["Birth_rate_crude"] = regressor_by_country_reset_index["Birth_rate_crude"].astype(float)
regressor_by_country_reset_index['Gini_index'].replace('..', np.nan, inplace=True)
regressor_by_country_reset_index["Gini_index"] = regressor_by_country_reset_index["Gini_index"].astype(float)
regressor_by_country_reset_index

Series Name,Year,Population_total,GDP_per_capita,UR,Urban_population,Birth_rate_crude,Gini_index
0,1960,445954579,83.035102,NaN,17.924,42.506,NaN
1,1961,456351876,85.969704,NaN,18.032,42.294,NaN
2,1962,467024193,90.276869,NaN,18.218,42.101,NaN
3,1963,477933619,101.315165,NaN,18.405,41.907,NaN
4,1964,489059309,115.487608,NaN,18.595,41.721,NaN
...,...,...,...,...,...,...,...
58,2018,1369003306,1974.377730,7.65,34.030,17.651,34.6
59,2019,1383112050,2050.163790,6.51,34.472,17.049,35.7
60,2020,1396387127,1913.219721,7.86,34.926,16.572,NaN
61,2021,1407563842,2238.127139,6.38,35.393,16.419,NaN


In [20]:
regressor_by_country = regressor_by_country_reset_index
regressor_by_country

Series Name,Year,Population_total,GDP_per_capita,UR,Urban_population,Birth_rate_crude,Gini_index
0,1960,445954579,83.035102,NaN,17.924,42.506,NaN
1,1961,456351876,85.969704,NaN,18.032,42.294,NaN
2,1962,467024193,90.276869,NaN,18.218,42.101,NaN
3,1963,477933619,101.315165,NaN,18.405,41.907,NaN
4,1964,489059309,115.487608,NaN,18.595,41.721,NaN
...,...,...,...,...,...,...,...
58,2018,1369003306,1974.377730,7.65,34.030,17.651,34.6
59,2019,1383112050,2050.163790,6.51,34.472,17.049,35.7
60,2020,1396387127,1913.219721,7.86,34.926,16.572,NaN
61,2021,1407563842,2238.127139,6.38,35.393,16.419,NaN


## Cleaning regressor by state dataset

In [21]:
#Read the csv
variables_by_state_raw = pd.read_csv("Resources/variable 2011-2021.csv")
variables_by_state_raw.head()

,State,Year,GDP,Birth Rate,GDP Per Capita,Unemployeement Rate
0,Alabama,2011,187605.8,1.84,85199.67147,0.095791
1,Alabama,2012,189245.5,1.81,86846.71580,0.081666
2,Alabama,2013,191369.8,1.79,88069.42997,0.073255
3,Alabama,2014,189886.3,1.83,87723.68787,0.067384
4,Alabama,2015,191335.2,1.83,88889.84213,0.061201


In [22]:
#Check the data type
variables_by_state_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   State                561 non-null    object 
 1   Year                 561 non-null    int64  
 2   GDP                  561 non-null    float64
 3   Birth Rate           561 non-null    float64
 4   GDP Per Capita       561 non-null    float64
 5   Unemployeement Rate  561 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 26.4+ KB


In [23]:
#Check the name of the states
states_unique = variables_by_state_raw["State"].unique()
states_unique

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [24]:
# Change name columns
variables_by_state_rename = variables_by_state_raw.rename(columns={"Unemployeement Rate": "UR_STATE",
                                                                   "GDP Per Capita":"GDP_per_capita_state", "Birth Rate":"Bith_rate_state"  })
variables_by_state_rename

,State,Year,GDP,Bith_rate_state,GDP_per_capita_state,UR_STATE
0,Alabama,2011,187605.8,1.84,85199.67147,0.095791
1,Alabama,2012,189245.5,1.81,86846.71580,0.081666
2,Alabama,2013,191369.8,1.79,88069.42997,0.073255
3,Alabama,2014,189886.3,1.83,87723.68787,0.067384
4,Alabama,2015,191335.2,1.83,88889.84213,0.061201
...,...,...,...,...,...,...
556,Wyoming,2017,37521.6,1.86,127693.43990,0.043294
557,Wyoming,2018,38080.1,1.79,130146.21760,0.040542
558,Wyoming,2019,38446.6,1.80,131049.22790,0.037222
559,Wyoming,2020,36268.6,1.69,123473.56420,0.059058


In [25]:
# Multiply the "unemployment rate" column by 10
variables_by_state_rename["UR_STATE"] *= 10
variables_by_state_rename

,State,Year,GDP,Bith_rate_state,GDP_per_capita_state,UR_STATE
0,Alabama,2011,187605.8,1.84,85199.67147,0.957912
1,Alabama,2012,189245.5,1.81,86846.71580,0.816665
2,Alabama,2013,191369.8,1.79,88069.42997,0.732545
3,Alabama,2014,189886.3,1.83,87723.68787,0.673836
4,Alabama,2015,191335.2,1.83,88889.84213,0.612008
...,...,...,...,...,...,...
556,Wyoming,2017,37521.6,1.86,127693.43990,0.432944
557,Wyoming,2018,38080.1,1.79,130146.21760,0.405421
558,Wyoming,2019,38446.6,1.80,131049.22790,0.372220
559,Wyoming,2020,36268.6,1.69,123473.56420,0.590579


### Get the regressors DataFrame per state.

In [26]:
#Filter by state
state_california_test = variables_by_state_rename[variables_by_state_rename["State"]=="California"]
state_california_test

,State,Year,GDP,Bith_rate_state,GDP_per_capita_state,UR_STATE
44,California,2011,2063827.5,1.90,112132.9778,1.188176
45,California,2012,2113096.4,1.89,114289.7100,1.052182
46,California,2013,2179229.0,1.84,117361.7150,0.902847
47,California,2014,2256054.7,1.84,120792.5513,0.756154
48,California,2015,2357452.9,1.79,125230.8652,0.624330
49,California,2016,2427894.6,1.77,127690.4670,0.551604
50,California,2017,2538204.0,1.69,132311.1528,0.483733
51,California,2018,2644061.2,1.63,137079.3844,0.425787
52,California,2019,2729225.8,1.60,140572.5763,0.409354
53,California,2020,2667220.9,1.52,140630.8786,1.014593


In [27]:
state_california_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 44 to 54
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   State                 11 non-null     object 
 1   Year                  11 non-null     int64  
 2   GDP                   11 non-null     float64
 3   Bith_rate_state       11 non-null     float64
 4   GDP_per_capita_state  11 non-null     float64
 5   UR_STATE              11 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 616.0+ bytes


In [28]:
regressor_by_state = state_california_test.drop(columns= "State").reset_index(drop=True)
regressor_by_state

,Year,GDP,Bith_rate_state,GDP_per_capita_state,UR_STATE
0,2011,2063827.5,1.90,112132.9778,1.188176
1,2012,2113096.4,1.89,114289.7100,1.052182
2,2013,2179229.0,1.84,117361.7150,0.902847
3,2014,2256054.7,1.84,120792.5513,0.756154
4,2015,2357452.9,1.79,125230.8652,0.624330
5,2016,2427894.6,1.77,127690.4670,0.551604
6,2017,2538204.0,1.69,132311.1528,0.483733
7,2018,2644061.2,1.63,137079.3844,0.425787
8,2019,2729225.8,1.60,140572.5763,0.409354
9,2020,2667220.9,1.52,140630.8786,1.014593


## Prepare DataFrames to merge

In [29]:
# DataFrame with the inmigration data
inmigration_us_df

,Year,Total_inmigrants,California
3,2005,84681,14724
12,2006,61369,14110
21,2007,65353,12820
30,2008,63352,14112
39,2009,57304,12826
48,2010,69162,15099
57,2011,69013,15061
66,2012,66434,13951
75,2013,68458,13735
84,2014,77908,15770


In [30]:
inmigration_us_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 3 to 147
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Year              17 non-null     int64
 1   Total_inmigrants  17 non-null     int64
 2   California        17 non-null     int64
dtypes: int64(3)
memory usage: 544.0 bytes


In [31]:
# Display regressor DataFrame per country
regressor_by_country

Series Name,Year,Population_total,GDP_per_capita,UR,Urban_population,Birth_rate_crude,Gini_index
0,1960,445954579,83.035102,NaN,17.924,42.506,NaN
1,1961,456351876,85.969704,NaN,18.032,42.294,NaN
2,1962,467024193,90.276869,NaN,18.218,42.101,NaN
3,1963,477933619,101.315165,NaN,18.405,41.907,NaN
4,1964,489059309,115.487608,NaN,18.595,41.721,NaN
...,...,...,...,...,...,...,...
58,2018,1369003306,1974.377730,7.65,34.030,17.651,34.6
59,2019,1383112050,2050.163790,6.51,34.472,17.049,35.7
60,2020,1396387127,1913.219721,7.86,34.926,16.572,NaN
61,2021,1407563842,2238.127139,6.38,35.393,16.419,NaN


In [32]:
regressor_by_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              63 non-null     int32  
 1   Population_total  63 non-null     int32  
 2   GDP_per_capita    63 non-null     float64
 3   UR                10 non-null     float64
 4   Urban_population  63 non-null     float64
 5   Birth_rate_crude  62 non-null     float64
 6   Gini_index        12 non-null     float64
dtypes: float64(5), int32(2)
memory usage: 3.1 KB


In [33]:
# Display regressor DataFrame per state
regressor_by_state

,Year,GDP,Bith_rate_state,GDP_per_capita_state,UR_STATE
0,2011,2063827.5,1.90,112132.9778,1.188176
1,2012,2113096.4,1.89,114289.7100,1.052182
2,2013,2179229.0,1.84,117361.7150,0.902847
3,2014,2256054.7,1.84,120792.5513,0.756154
4,2015,2357452.9,1.79,125230.8652,0.624330
5,2016,2427894.6,1.77,127690.4670,0.551604
6,2017,2538204.0,1.69,132311.1528,0.483733
7,2018,2644061.2,1.63,137079.3844,0.425787
8,2019,2729225.8,1.60,140572.5763,0.409354
9,2020,2667220.9,1.52,140630.8786,1.014593


In [44]:
# Get final DataFrame

# First merge inmigration_us_df and regressor_by_country
df_merged1 = pd.merge(inmigration_us_df, regressor_by_country, on="Year", how="left")

# Then merge the result with regressor_by_state
df_merged = pd.merge(df_merged1, regressor_by_state, on="Year", how="left")

df_merged


,Year,Total_inmigrants,California,Population_total,GDP_per_capita,UR,Urban_population,Birth_rate_crude,Gini_index,GDP,Bith_rate_state,GDP_per_capita_state,UR_STATE
0,2005,84681,14724,1154638713,710.509345,8.70,29.235,23.940,NaN,NaN,NaN,NaN,NaN
1,2006,61369,14110,1172373788,802.013742,NaN,29.569,23.220,NaN,NaN,NaN,NaN,NaN
2,2007,65353,12820,1189691809,1022.732476,NaN,29.906,22.713,NaN,NaN,NaN,NaN,NaN
3,2008,63352,14112,1206734806,993.503412,NaN,30.246,22.276,NaN,NaN,NaN,NaN,NaN
4,2009,57304,12826,1223640160,1096.636136,NaN,30.587,21.934,35.4,NaN,NaN,NaN,NaN
5,2010,69162,15099,1240613620,1350.634457,3.12,30.930,21.438,NaN,NaN,NaN,NaN,NaN
6,2011,69013,15061,1257621191,1449.603301,NaN,31.276,20.945,35.7,2063827.5,1.90,112132.9778,1.188176
7,2012,66434,13951,1274487215,1434.017978,3.22,31.634,20.421,NaN,2113096.4,1.89,114289.7100,1.052182
8,2013,68458,13735,1291132063,1438.056995,NaN,32.003,19.935,NaN,2179229.0,1.84,117361.7150,0.902847
9,2014,77908,15770,1307246509,1559.863772,NaN,32.384,19.049,NaN,2256054.7,1.84,120792.5513,0.756154


In [45]:
columns_merge = df_merged.columns
columns_merge

Index(['Year', 'Total_inmigrants', 'California', 'Population_total',
       'GDP_per_capita', 'UR', 'Urban_population', 'Birth_rate_crude',
       'Gini_index', 'GDP', 'Bith_rate_state', 'GDP_per_capita_state',
       'UR_STATE'],
      dtype='object')

## Prediction with Prophet

In [46]:
from prophet import Prophet

In [47]:
# Handle NaN values in the DataFrame
# For simplicity, we'll fill the NaN values with the column means
df_merged.fillna(df_merged.mean(), inplace=True)

In [48]:

from sklearn.impute import SimpleImputer

# Step 1: Prepare the DataFrame with time series data and regressors
prophet_df = df_merged[['Year', 'California', 'Population_total', 'GDP_per_capita', 'UR', 'Urban_population', 'Birth_rate_crude', 'Gini_index', 'GDP', 'GDP_per_capita_state']].copy()
prophet_df.rename(columns={'Year': 'ds', 'California': 'y'}, inplace=True)
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'], format='%Y')

# Step 2: Handle missing values in the DataFrame
imputer = SimpleImputer(strategy='mean')
prophet_df[['Population_total', 'GDP_per_capita', 'UR', 'Urban_population', 'Gini_index', 'GDP', 'GDP_per_capita_state']] = imputer.fit_transform(prophet_df[['Population_total', 'GDP_per_capita', 'UR', 'Urban_population', 'Gini_index', 'GDP',  'GDP_per_capita_state']])

prophet_df_2=prophet_df.iloc[0:9].copy()

display(prophet_df_2)

# Separate the target variable 'y' and other features
X = prophet_df_2.drop(columns=['y','ds'])
y = prophet_df_2['y']

# Create a Random Forest Regressor model
rf = RandomForestRegressor(random_state=42)
rf.fit(X, y)

# Get the feature importances
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': rf.feature_importances_})

# Sort the features by importance in descending order
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Display the feature importances
print(feature_importances)


top_n_features = feature_importances['Feature'][:3].tolist()




prophet_df_pred=prophet_df[:-1].copy()

prophet_df_pred['y']=np.log1p(prophet_df_pred['y'])

# Step 3: Set up the Prophet model with additional regressors
holidays = pd.DataFrame({
    'holiday': '2020',
    'ds': pd.to_datetime(['2020-01-01']),
    'lower_window': 0,
    'upper_window': 0,
})

model = Prophet(holidays=holidays)

for feature in top_n_features:
    model.add_regressor(feature)


# Step 4: Fit the model to your data
model.fit(prophet_df_pred)

display(prophet_df_pred)
   




,ds,y,Population_total,GDP_per_capita,UR,Urban_population,Birth_rate_crude,Gini_index,GDP,GDP_per_capita_state
0,2005-01-01,14724,1.154639e+09,710.509345,8.700000,29.235,23.940,35.257143,2440999.8,129052.399991
1,2006-01-01,14110,1.172374e+09,802.013742,6.205714,29.569,23.220,35.257143,2440999.8,129052.399991
2,2007-01-01,12820,1.189692e+09,1022.732476,6.205714,29.906,22.713,35.257143,2440999.8,129052.399991
3,2008-01-01,14112,1.206735e+09,993.503412,6.205714,30.246,22.276,35.257143,2440999.8,129052.399991
4,2009-01-01,12826,1.223640e+09,1096.636136,6.205714,30.587,21.934,35.400000,2440999.8,129052.399991
5,2010-01-01,15099,1.240614e+09,1350.634457,3.120000,30.930,21.438,35.257143,2440999.8,129052.399991
6,2011-01-01,15061,1.257621e+09,1449.603301,6.205714,31.276,20.945,35.700000,2063827.5,112132.977800
7,2012-01-01,13951,1.274487e+09,1434.017978,3.220000,31.634,20.421,35.257143,2113096.4,114289.710000
8,2013-01-01,13735,1.291132e+09,1438.056995,6.205714,32.003,19.935,35.257143,2179229.0,117361.715000


19:35:40 - cmdstanpy - INFO - Chain [1] start processing


                Feature  Importance
1        GDP_per_capita    0.251828
5            Gini_index    0.140628
4      Birth_rate_crude    0.123912
0      Population_total    0.122526
2                    UR    0.105133
3      Urban_population    0.103833
6                   GDP    0.085697
7  GDP_per_capita_state    0.066442


19:35:40 - cmdstanpy - INFO - Chain [1] done processing


,ds,y,Population_total,GDP_per_capita,UR,Urban_population,Birth_rate_crude,Gini_index,GDP,GDP_per_capita_state
0,2005-01-01,9.597302,1.154639e+09,710.509345,8.700000,29.235,23.940,35.257143,2440999.8,129052.399991
1,2006-01-01,9.554710,1.172374e+09,802.013742,6.205714,29.569,23.220,35.257143,2440999.8,129052.399991
2,2007-01-01,9.458840,1.189692e+09,1022.732476,6.205714,29.906,22.713,35.257143,2440999.8,129052.399991
3,2008-01-01,9.554852,1.206735e+09,993.503412,6.205714,30.246,22.276,35.257143,2440999.8,129052.399991
4,2009-01-01,9.459308,1.223640e+09,1096.636136,6.205714,30.587,21.934,35.400000,2440999.8,129052.399991
5,2010-01-01,9.622450,1.240614e+09,1350.634457,3.120000,30.930,21.438,35.257143,2440999.8,129052.399991
6,2011-01-01,9.619930,1.257621e+09,1449.603301,6.205714,31.276,20.945,35.700000,2063827.5,112132.977800
7,2012-01-01,9.543378,1.274487e+09,1434.017978,3.220000,31.634,20.421,35.257143,2113096.4,114289.710000
8,2013-01-01,9.527775,1.291132e+09,1438.056995,6.205714,32.003,19.935,35.257143,2179229.0,117361.715000
9,2014-01-01,9.665928,1.307247e+09,1559.863772,6.205714,32.384,19.049,35.257143,2256054.7,120792.551300


In [52]:
regressors_row_2021 = prophet_df.iloc[-1, :][top_n_features].to_dict()

future_dates=pd.DataFrame({'ds': pd.to_datetime(['2021'], format='%Y')})


# Step 5: Create a DataFrame with the future dates for prediction (e.g., for 2021)
for feature in top_n_features:
    future_dates[feature] = regressors_row_2021[feature]
    
    
future_dates["ds"]=pd.to_datetime('2021', format='%Y')

display(future_dates) 

# Step 6: Predict the number of immigrants for the future dates
forecast = model.predict(future_dates)

# Step 7: Extract the predicted values for the future dates
predicted_values = np.exp(forecast['yhat'])

# Step 8: Display the predicted values for 2021
print("Predicted value "+str(predicted_values))


from sklearn.metrics import mean_absolute_error
y_true=df_merged['California'][-1:].values
print("Real:"+str(y_true))
y_pred=np.exp(forecast['yhat'].values)
print("Predicted:"+str(y_pred))
mae=mean_absolute_error(y_true,y_pred)
print("The MAE while testing 2021: "+str(mae))

#mae = mean_absolute_error(y_true, y_pred)
#mse = mean_squared_error(y_true, y_pred)
#rmse = np.sqrt(mse)
#r2 = r2_score(y_true, y_pred)

#print("Mean Absolute Error (MAE):", mae)
#print("Mean Squared Error (MSE):", mse)
#print("Root Mean Squared Error (RMSE):", rmse)
#print("R-squared (R2):", r2)



,ds,GDP_per_capita,Gini_index,Birth_rate_crude
0,2021-01-01,2238.127139,35.257143,16.419


Predicted value 0    10229.48471
Name: yhat, dtype: float64
Real:[17299]
Predicted:[10229.48470969]
The MAE while testing 2021: 7069.515290306557
Mean Absolute Error (MAE): 7069.515290306557
Mean Squared Error (MSE): 49978046.43987821
Root Mean Squared Error (RMSE): 7069.515290306557
R-squared (R2): nan


C:\Users\MIRIAM\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [50]:



future_dates["ds"]=pd.to_datetime('2023', format='%Y')

forecast=model.predict(future_dates)
predicted_values = forecast[['ds', 'yhat']]
print(predicted_values)
print(np.exp(forecast['yhat'].values))

import matplotlib.pyplot as plt

#fig=model.plot(forecast)
#plt.scatter(pd.to_datetime(['2023'], format='%Y'), forecast['yhat'], c='r', s=100, label='2023 Prediction')
#plt.legend()
#plt.show()





          ds      yhat
0 2023-01-01  9.204798
[9944.7321594]


In [42]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Extract the predicted values for the test period
predicted_values = forecast[['ds', 'yhat']].tail(1)['yhat'].values

# Calculate evaluation metrics
mae = mean_absolute_error(actual_values, predicted_values)
mse = mean_squared_error(actual_values, predicted_values)
rmse = np.sqrt(mse)
r2 = r2_score(actual_values, predicted_values)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)


Mean Absolute Error (MAE): 17298.124098778484
Mean Squared Error (MSE): 299225097.3367409
Root Mean Squared Error (RMSE): 17298.124098778484
R-squared (R2): nan


C:\Users\MIRIAM\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
